In [1]:
from SplitWindows import SplitWindows
from AttentionArea import TemporalAttention , SpatialAttention , DynamicMatrix , AttentionBlock
from torch_geometric.nn import GCNConv
from Config import CONFIG
from Dataset import TimeSDataset
import engine

import numpy as np
import pandas as pd 
import torch
from torch import nn
from torch_geometric.nn import GCNConv
from torch.utils.data import DataLoader

Config = CONFIG()
device = "cuda" if torch.cuda.is_available else "cpu"

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Config.T

23

In [3]:
### Write the Main Class

class DynDeepNet(nn.Module):
    def __init__(self ):
        super().__init__()
        # Define Clf token
        self.CLFToken = torch.rand(1, Config.T_prim, 1)
        # Define Temporal Attention
        self.TAttention = TemporalAttention (Config.NUM_H ,Config.V * Config.K_E , Config.HIDDEN_DIM)
        # Define Spatial Attention 
        self.SAttention = SpatialAttention()
        # Define Dynamic Matrix
        self.Dm = DynamicMatrix()
        # Define GCN Layer
        self.gcn_layer = GCNConv(in_channels=Config.K_E, out_channels=Config.K_F)
        # Define Final Encoder layers
        self.Transformer = nn.Sequential(*[AttentionBlock(Config.T * Config.K_F, Config.HIDDEN_DIM, Config.NUM_H, Config.DROP_OUT) for _ in range(Config.NUM_LAYER)])
        self.Mlp_head = nn.Sequential(
            nn.LayerNorm(Config.T * Config.K_F),
            nn.Linear(Config.T * Config.K_F, Config.NUM_CLASS)
            # torch.nn.Softmax(dim=1)
        )
        self.Dropout = nn.Dropout(Config.DROP_OUT)
        self.Flatten = nn.Flatten(2,-1)
    def forward(self,x):
        ### Input : (B , T_Prim , V)
        B , _ , _ = x.shape
        ## ADD CLF TOKEN
        CLFToken = self.CLFToken.repeat(B, 1 ,1).to(device)
        x = torch.concatenate([CLFToken, x], axis=-1)
        
        ## Convert Signal Serie to the Window Signal
        x = SplitWindows(x)  ## OutPut : (B , T , V ,P)
        ## Temporal Transformer
        x = self.TAttention(x)  ## OutPut : (B , T , V , K_E)
        ## Spatial Attention
        x = self.SAttention(x)    ## OutPut : (B , T , V , K_E)
        # Apply Drop out
        x = self.Dropout(x)
        ## Dynamic Matrix
        A = self.Dm(x)           ## OutPut  : (B , T , V , V)
        ## GCN Layer
        BatchList = []
        # Iterate over each graph in the batch
        for i in range(B):
            # Extract the adjacency matrix for the current graph
            adj_i = A[i]  # Shape (T, V, V)
            out = []
            for t in range(Config.T):
                # Get the edge indices and weights for the current time step
                edge_index = adj_i[t].nonzero(as_tuple=False).T  # Shape (2, E)
                edge_weights = adj_i[t][edge_index[0], edge_index[1]]  # Shape (E,)
                # Select the corresponding node features for the current time step
                x_t = x[i][t]
                # Apply the GCN layer
                x_t = self.gcn_layer(x_t, edge_index, edge_weight=edge_weights)
                # Store the output for the current time step
                out.append(x_t)
            BatchList.append(torch.stack(out))
        
        # Stack the output to get the final representation
        x = torch.stack(BatchList)  ## OUTPUT : (B , T , V , K_F) 
        # Transpose
        x = x.transpose(1,2)  ## OUTPUT : (B , V , T , K_F)
        # Flatten
        x = self.Flatten(x)  ## OUTPUT  : (B , V , T * K_F)
        # Drop Out
        x = self.Dropout(x)
        # Apply Final Encoder
        x = x.transpose(0,1)
        x = self.Transformer(x)   ## OUTPUT : (V , B , T * K_F)
        # Apply Linear Head
        cls = x[0]
        return self.Mlp_head(cls)

Model = DynDeepNet()
        

## Define Dataset & DataLoaders

In [4]:
TrainDataset = TimeSDataset("../Data/AAL/Train","Gender",label_path="../Data/Behavioral-HCP.csv")
TestDataset = TimeSDataset("../Data/AAL/Test","Gender",label_path="../Data/Behavioral-HCP.csv")

In [5]:
TrainDataLoader = DataLoader(TrainDataset ,Config.B ,shuffle = True ,num_workers= 2)
TestDataLoader = DataLoader(TestDataset ,Config.B ,shuffle = False ,num_workers= 2)

## Train the model

In [6]:
### 
LR = 0.001
Pivot = 80
Part = 1
Epochs = 20
save_weights = False
###
optimizer = torch.optim.AdamW(Model.parameters() , LR)
loss_fn = torch.nn.CrossEntropyLoss()

In [7]:
results = engine.train(Model , 
                TrainDataLoader,
                TestDataLoader,
                optimizer,
                loss_fn,
                Pivot,
                Part,
                save_weights,
                Epochs,
                device)

  0%|                                                                                                           | 0/20 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1004.00 MiB. GPU 0 has a total capacty of 3.80 GiB of which 839.81 MiB is free. Process 11753 has 2.47 GiB memory in use. Of the allocated memory 2.33 GiB is allocated by PyTorch, and 43.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF